# Are we 3.12 yet?

Exploring the `conda-forge` [`python312` migration](https://conda-forge.org/status/#python312).

In [ ]:
import sys
if "pyodide" in sys.modules and __name__ == "__main__":
    %pip install -r requirements.txt

In [ ]:
if "pyodide" in sys.modules:
    __import__("pyodide_http").patch_all()

In [ ]:
import pandas as pd
from ipyforcegraph import graphs as G, behaviors as B
from ipyforcegraph.behaviors import scales as S 
import ipylab
import requests

In [ ]:
URL = "https://raw.githubusercontent.com/regro/cf-graph-countyfair/master/status/migration_json/python312.json"
data = requests.get(URL).json()
data.keys()
df = pd.DataFrame([{"id": name, **d}for name, d in data.pop("_feedstock_status").items()]).fillna("")
df.sample(10).T

In [ ]:
links = []
df.apply(lambda d: links.extend({"source": d.id, "target": t} for t in d.immediate_children), axis=1)
links = pd.DataFrame(links)
links.T

In [ ]:
dk = [*data.keys()]
nodes = pd.concat([df, pd.DataFrame(set(links["target"]) - set(df["id"]), columns=["id"])]).fillna("")
nodes["status"] = df.apply(lambda d: [k for k in data if d.id in data[k]][-1], axis=1)
nodes["status_color"] = df.apply(lambda d: [dk.index(k) for k in data if d.id in data[k]][-1], axis=1)

In [ ]:
nodes.T

In [ ]:
fg = G.ForceGraph3D(
    source=G.DataFrameSource(nodes=nodes, links=links), 
    layout=dict(min_height="80vh")
)

In [ ]:
if __name__ == "__main__":
    app = ipylab.JupyterFrontEnd()
    panel = ipylab.Panel([fg])
    app.shell.add(panel, "main", {"mode": "split-right"})

pretty

In [ ]:
c = S.ContinuousColor(
    "status_color", scheme=S.ContinuousColor.Scheme.viridis, domain=[-2, 5]
)
s = 4 # B.Nunjucks("""{{ node.status_color * 100 }}""")
ns = B.NodeShapes(color=B.CaptureAs("_color", c), size=s)
tip = B.NodeTooltip(B.Nunjucks("""
<h1>{{ node.id }}</h1> ({{ node.status }})
{{ node.immediate_children }}
"""), color="black")
fg.behaviors=[ns, tip]